In [1]:
import numpy as np
import pandas as pd 
import scipy.sparse 
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet,Ridge,LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)
sns.set(rc={'figure.figsize':(20, 10)})

In [2]:
import gc
from tqdm import tqdm_notebook
def get_X_y_ensembling(df,clip=20):
    y = np.clip(df.target.values,0,clip)
    X = df.drop(['target','date_block_num'],axis=1)
    return X,y
def get_oof_pickle(filename,clip=20):
    return np.clip(pd.read_pickle("journey/data/"+filename),0,clip)
def get_all_data(data_path,filename):
    all_data = pd.read_pickle(data_path + filename)
    all_data = downcast_dtypes(all_data)
    all_data = all_data.reset_index().drop('index',axis=1)
    return all_data


def downcast_dtypes(df):
    '''
    Changes column types in the dataframe: 

        `float64` type to `float32`
        `int64`   type to `int32`
    '''

    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    object_cols =   [c for c in df if df[c].dtype == "object"]
    # Downcast
    for i in float_cols:
        df[i] = df[i].astype(np.float32)
    for j in int_cols  : 
        df[j]   = df[j].astype(np.int32)
    for j in object_cols  : 
        df[j]   = df[j].astype(str).astype(int)

    return df


def rmse(*args):
    
    """ Funcion that calculates the root mean squared error"""
    return np.sqrt(mean_squared_error(*args))


def clip20(x):
    return np.clip(x, 0, 20)

def clip40(x):
    return np.clip(x, 0, 20)

def get_cv_idxs(df,start,end):
    result=[]
    for i in range(start,end+1):
        dates = df.date_block_num
        train_idx = np.array(df.loc[dates <i].index)
        val_idx = np.array(df.loc[dates == i].index)
        result.append((train_idx,val_idx))
    return np.array(result)
def get_submission(item_cnt_month,sub_name,clip=20 ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv('journey/data/test.csv')
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv( sub_name+'.csv',index=False)
    return sub
def root_mean_squared_error(truth,pred):
    return sqrt(mean_squared_error(truth,pred))

In [3]:
to_drop_cols=['item__target_mean',"item_id",
 'shop__target_mean','item_category_id',#"shop_id",
 'item_mean','is_holiday','is_business_day','is_weekend',
 'target_mean',#"item_time",
 'target',#"city_code",
 'date_block_num','year_min',#"month_min" 
 'item_cnt_day_year_shop_id', 
  #'date_block_num_min',
 'item_cnt_day_mean_new', 'item_cnt_day_shop_id',
 'item_cnt_day_year_shop_id', 'item_cnt_day_cats_id',
'item_cnt_day_year_cats_id','item_cnt_day_new_shop','item_cnt_day_new_cats']

In [4]:
lgb_oof = get_oof_pickle('lgb_best.pickle')

lgb2_oof = get_oof_pickle('lgb_best2.pickle')
#xgb3_oof = get_oof_pickle('xgb3.pickle')

#fast_oof = get_oof_pickle('xgb.pickle')

all_data = get_all_data("journey/data/",'new_train.pickle')

all_data.target = np.clip(all_data.target.values,0,20)

temp = [lgb_oof,lgb2_oof,#xgb3_oof,
        #fast_oof,
        lgb_oof-lgb2_oof,#lgb_oof-xgb3_oof,
         #lgb_oof-fast_oof,
         #lgb2_oof-xgb3_oof,
        #lgb2_oof-fast_oof,#xgb3_oof-fast_oof,
         #np.mean(np.stack([lgb_oof,lgb2_oof,xgb3_oof,fast_oof],axis=1),axis=1),
         np.mean(np.stack([lgb_oof,lgb2_oof],axis=1),axis=1)]

all28 = all_data[all_data.date_block_num>=28].copy()

all_oof = np.stack(temp+[all28.target_lag_1,all28.date_block_num,all28.target],axis=1)

all_oof_df = pd.DataFrame(all_oof,columns=['lgb','lgb2',#'xgb3',
                                            #'fast',
                                            'lgb_lgb2',#'lgb_xgb3',
                                            #'lgb_fast',
                                            #'lgb2_xgb3',
                                            #'lgb2_fast',#'xgb3_fast',
                                            'oof_mean','target_lag_1','date_block_num','target'])

all_oof_df.to_pickle("journey/data/"+'ensembling_nofast1.pickle')

In [5]:
all_oof_df= pd.read_pickle("journey/data/"+'ensembling_nofast1.pickle')
all_oof_df.head().T

,0,1,2,3,4
lgb,0.119366,0.034570,0.041229,0.852336,1.871450
lgb2,0.105166,0.038287,0.054302,0.851417,1.413097
lgb_lgb2,0.014200,-0.003717,-0.013073,0.000920,0.458354
oof_mean,0.112266,0.036429,0.047765,0.851877,1.642273
target_lag_1,0.000000,0.000000,0.000000,1.000000,3.000000
date_block_num,28.000000,28.000000,28.000000,28.000000,28.000000
target,2.000000,1.000000,1.000000,2.000000,2.000000


In [6]:
cv = get_cv_idxs(all_oof_df,29,33)

In [9]:
lgb_pred = pd.read_csv('journey/data/lgb_new.csv').item_cnt_month
lgb2_pred = pd.read_csv('journey/data/lgb_new2.csv').item_cnt_month
#xgb3_pred = pd.read_csv('journey/data/xgb2.csv').item_cnt_month
#fast_pred = pd.read_csv('journey/data/xgb.csv').item_cnt_month

target_lag_1 = pd.read_pickle('journey/data/sub_data.pickle').target_lag_1

temp = [lgb_pred,lgb2_pred,
        #xgb3_pred,
        #fast_pred,
        lgb_pred-lgb2_pred,
        #lgb_pred-xgb3_pred,
        #lgb_pred-fast_pred,
        #lgb2_pred-xgb3_pred,
        #lgb2_pred-fast_pred,#xgb3_pred-fast_pred,
        #np.mean(np.stack([lgb_pred,lgb2_pred,xgb3_pred,fast_pred],axis=1),axis=1)
        np.mean(np.stack([lgb_pred,lgb2_pred],axis=1),axis=1),target_lag_1]

temp = np.stack(temp,axis=1)
temp.shape

test_df = pd.DataFrame(temp,columns=['lgb','lgb2',#'xgb3',
                                           #'fast',
                                           'lgb_lgb2',#'lgb_xgb3',
                                            #'lgb_fast',
                                           #'lgb2_xgb3',
                                           #'lgb2_fast',#'xgb3_fast',
                                           'oof_mean','target_lag_1'])

test_df.to_csv('journey/data/ensembling_test_nofast.csv',index=False)

In [10]:
test_df= pd.read_csv('journey/data/ensembling_test_nofast.csv')
test_df.head()

,lgb,lgb2,lgb_lgb2,oof_mean,target_lag_1
0,0.851010,0.514649,0.336361,0.682829,0.0
1,0.302321,0.289921,0.012400,0.296121,0.0
2,1.851504,1.369511,0.481993,1.610507,1.0
3,0.454370,0.246145,0.208225,0.350258,0.0
4,3.425584,3.879200,-0.453616,3.652392,0.0


In [9]:
final_pred = lgb_pred*0.1 + lgb2_pred*0.8+fast_pred*0.1

get_submission(final_pred,'ensembling_average'); 

In [11]:
X,y = get_X_y_ensembling(all_oof_df)


lr = LinearRegression()
params={
     #'l1_ratio': [.01,0.05,.1, .7, 1],
    'fit_intercept':[True,False],
    'normalize':[True,False]
}
clf=  GridSearchCV(lr,params,cv=cv,scoring = make_scorer(root_mean_squared_error,greater_is_better=False),verbose=True,n_jobs=-1)

In [ ]:
clf.fit(X,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [ ]:
print(clf.best_params_)
print(clf.best_score_)

In [11]:
X,y = get_X_y_ensembling(all_oof_df)

lr = LinearRegression(fit_intercept=False,n_jobs=-1)

lr.fit(X,y)

X.head()

test_pred =  lr.predict(test_df)

pd.Series(test_pred).describe()

get_submission(test_pred,'ensembling_lr')

,ID,item_cnt_month
0,0,0.651706
1,1,0.280271
2,2,1.533338
3,3,0.334919
4,4,3.446081
5,5,0.970332
6,6,0.763240
7,7,0.096840
8,8,1.060670
9,9,0.053534


In [11]:
X,y = get_X_y_ensembling(all_oof_df)

# elastic = ElasticNetCV(random_state=1402,l1_ratio=[.01,0.05,.1, .5, .7, .9, .95, .99, 1],cv =cv,verbose=10,n_jobs=2)
# elastic = ElasticNet(random_state=1402)
lr = Ridge(random_state=1402)
params={
#     'l1_ratio': [0.0,.001,.005,.01,0.05,.1, .5,.6, .7, .8,.9, 1]
    
    'alpha':[0.0,0.05,.1, .5],
    'fit_intercept': [False],
    'solver':['svd','cholesky','sparse_cg','lsqr','sag']
    
}
clf=  GridSearchCV(lr,params,cv=cv,scoring = make_scorer(root_mean_squared_error,greater_is_better=False),verbose=True,n_jobs=-1)

In [ ]:
clf.fit(X,y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
print(clf.best_params_)
print(clf.best_score_)

In [ ]:
cv_result=pd.DataFrame(clf.cv_results_).sort_values('rank_test_score')
cv_result.T

In [12]:
X,y = get_X_y_ensembling(all_oof_df)

params={'alpha': 0.0, 'fit_intercept': False, 'solver': 'sag','random_state':1402}
lr = Ridge(**params)

lr.fit(X,y)

test_pred =  lr.predict(test_df)

pd.Series(test_pred).describe()

get_submission(test_pred,'ensembling_ridge')

,ID,item_cnt_month
0,0,0.648678
1,1,0.280654
2,2,1.530090
3,3,0.332915
4,4,3.458633
5,5,0.970291
6,6,0.767919
7,7,0.096771
8,8,1.062878
9,9,0.055054
